In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models

# Load dataset (assuming it's in CSV format)
data = pd.read_csv("Datasets/9 Credit Card Dataset/creditcard.csv")

# Separate features and target
X = data.drop('Class', axis=1)  # Features
y = data['Class']  # Target

# Standardize the features for better performance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets, using only normal transactions (Class 0) for training
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train = X_train[y_train == 0]  # Train only on non-fraudulent transactions (Class 0)
X_test = X_test  # Test set includes both fraud and non-fraud transactions
y_test = y_test


In [5]:
# Define the autoencoder architecture
input_dim = X_train.shape[1]  # Number of features

# Encoder
input_layer = layers.Input(shape=(input_dim,))
encoded = layers.Dense(16, activation="relu")(input_layer)
encoded = layers.Dense(8, activation="relu")(encoded)

# Decoder
decoded = layers.Dense(16, activation="relu")(encoded)
decoded = layers.Dense(input_dim, activation="sigmoid")(decoded)

# Full autoencoder model
autoencoder = models.Model(inputs=input_layer, outputs=decoded)

# Compile the model
autoencoder.compile(optimizer="adam", loss="mean_squared_error")

# Summary of the model
autoencoder.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 30)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,286 (5.02 KB)

 Trainable params: 1,286 (5.02 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Train the model
history = autoencoder.fit(
    X_train, X_train,
    epochs=50,
    batch_size=64,
    shuffle=True,
    validation_data=(X_test[y_test == 0], X_test[y_test == 0])  # Validate on normal transactions only
)


Epoch 1/50
3554/3554 ━━━━━━━━━━━━━━━━━━━━ 4s 741us/step - loss: 0.9120 - val_loss: 0.7719
Epoch 2/50
3554/3554 ━━━━━━━━━━━━━━━━━━━━ 3s 738us/step - loss: 0.7921 - val_loss: 0.7593
Epoch 3/50
3554/3554 ━━━━━━━━━━━━━━━━━━━━ 3s 739us/step - loss: 0.7634 - val_loss: 0.7512
Epoch 4/50
3554/3554 ━━━━━━━━━━━━━━━━━━━━ 3s 714us/step - loss: 0.7529 - val_loss: 0.7448
Epoch 5/50
3554/3554 ━━━━━━━━━━━━━━━━━━━━ 3s 716us/step - loss: 0.7496 - val_loss: 0.7416
Epoch 6/50
3554/3554 ━━━━━━━━━━━━━━━━━━━━ 3s 733us/step - loss: 0.7644 - val_loss: 0.7398
Epoch 7/50
3554/3554 ━━━━━━━━━━━━━━━━━━━━ 3s 726us/step - loss: 0.7506 - val_loss: 0.7379
Epoch 8/50
3554/3554 ━━━━━━━━━━━━━━━━━━━━ 3s 733us/step - loss: 0.7448 - val_loss: 0.7367
Epoch 9/50
3554/3554 ━━━━━━━━━━━━━━━━━━━━ 3s 718us/step - loss: 0.7640 - val_loss: 0.7360
Epoch 10/50
3554/3554 ━━━━━━━━━━━━━━━━━━━━ 3s 715us/step - loss: 0.7626 - val_loss: 0.7352
Epoch 11/50
3554/3554 ━━━━━━━━━━━━━━━━━━━━ 3s 708us/step - loss: 0.7447 - val_loss: 0.7343
Epoch 12

In [9]:
# Calculate reconstruction errors on the test set
reconstructions = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - reconstructions, 2), axis=1)

# Set the threshold for anomaly
threshold = np.percentile(mse, 95)  # Set threshold at 95th percentile of error on normal transactions
print(f"Reconstruction error threshold: {threshold}")


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 1s 558us/step
Reconstruction error threshold: 1.5264392217081386


In [11]:
# Detect anomalies
y_pred = mse > threshold
y_pred = y_pred.astype(int)  # Convert boolean to integer (1 = anomaly, 0 = normal)

# Evaluate the performance
from sklearn.metrics import classification_report, confusion_matrix

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Confusion Matrix:
[[54103  2761]
 [   10    88]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.98     56864
           1       0.03      0.90      0.06        98

    accuracy                           0.95     56962
   macro avg       0.52      0.92      0.52     56962
weighted avg       1.00      0.95      0.97     56962

